In [ ]:
#|default_exp dev_utils

In [ ]:
#| export
import re
import sys
import functools
from __future__ import annotations

In [ ]:
#| export
__all__ = ['stack_trace', 'stack_trace_jupyter']

In [ ]:
#| export
class StackTrace():
    """Capture and prints information on all stack frame executed"""
    def __init__(self, 
                 with_call:bool=True,      
                 with_return:bool=True, 
                 with_exception:bool=True, 
                 max_depth:int=-1
                ):
        self._frame_dict = {}
        self._options = set()
        self._max_depth = max_depth
        if with_call: self._options.add('call')
        if with_return: self._options.add('return')
        if with_exception: self._options.add('exception')

    def __call__(self, 
                 frame, 
                 event, 
                 arg
                ):
        ret = []
        co_name = frame.f_code.co_name
        co_filename = frame.f_code.co_filename
        co_lineno = frame.f_lineno
        if event == 'call':
            back_frame = frame.f_back
            if back_frame in self._frame_dict:
                self._frame_dict[frame] = self._frame_dict[back_frame] + 1
            else:
                self._frame_dict[frame] = 0

        depth = self._frame_dict[frame]

        if event in self._options and (self._max_depth<0 or depth <= self._max_depth):
            ret.append(co_name)
            ret.append(f'[{event}]')
            if event == 'return':
                ret.append(arg)
            elif event == 'exception':
                ret.append(repr(arg[0]))
            ret.append(f'in {co_filename} line:{co_lineno}')
        if ret:
            self.print_stack_info(co_filename, ret, depth)
        return self

    def print_stack_info(self, 
                         co_filename, 
                         ret, 
                         depth
                        ):
        """This methods can be overloaded to customize what is printed out"""
        text = '\t'.join([str(i) for i in ret])
        print(f"{'  ' * depth}{text}")

In [ ]:
#| export
class StackTraceJupyter(StackTrace):
    """Prints stack frame information in Jupyter notebook context (filters out jupyter overhead)"""

    def print_stack_info(self, 
                         co_filename, 
                         ret, 
                         depth
                        ):
        """Overload the base class to filter out those calls to Jupyter overhead functions"""

        EXCL_LIBS = ['encodings.*', 'ntpath.*', 'threading.*', 'weakref.*']
        EXCL_SITE_PACKAGES = ['colorama', 'ipykernel', 'zmq']

        PATH_TO_LIBS_RE = r'^[a-zA-Z]:\\([^<>:\"/\\|?\*]*)\\envs\\([^<>:\"/\\|?\*]*)\\lib'
        LIBS = f"{'|'.join(EXCL_LIBS)}"
        SITE_PACKAGES = f"{'|'.join(EXCL_SITE_PACKAGES)}"
        MODULE_FILTERS_RE = rf"{PATH_TO_LIBS_RE}\\(({LIBS})|(site-packages\\({SITE_PACKAGES}))\\.*)"

        pat = re.compile(MODULE_FILTERS_RE)
        match = pat.match(co_filename)
        
        if match is None:
            """Only print stack frame info for those objects where there is no match"""
            text = '\t'.join([str(i) for i in ret])
            print(f"{'  ' * depth}{text}")

In [ ]:
#| export
def stack_trace(**kw):
    """Function for stack_trace decorator"""
    def entangle(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # st = StackTrace(**kw)
            st = StackTrace(**kw)
            sys.settrace(st)
            try:
                return func(*args, **kwargs)
            finally:
                sys.settrace(None)
        return wrapper
    return entangle

In [ ]:
#| export
def stack_trace_jupyter(**kw):
    """Function for stack_trace_jupyter decorator"""
    def entangle(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # st = StackTrace(**kw)
            st = StackTraceJupyter(**kw)
            sys.settrace(st)
            try:
                return func(*args, **kwargs)
            finally:
                sys.settrace(None)
        return wrapper
    return entangle

## Usage:

Several functions, some of them nested and some of them with errors.

In [ ]:
def foo():
        pass

def bar():
    foo()
    return 0

def error():
    1/0

def recur(i):
    if i == 0:
        return
    recur(i-1)

Using the `@stack_trace` or `@stack_trace_jupyter` decorator allows a detailled view the trace function by function and where it fails.

In [ ]:
@stack_trace(with_return=True, with_exception=True, max_depth=3)
def test():
    bar()
    recur(5)
    error()
    
test()

test	[call]	in /tmp/ipykernel_28345/2975652090.py line:1
  bar	[call]	in /tmp/ipykernel_28345/2052304305.py line:4
    foo	[call]	in /tmp/ipykernel_28345/2052304305.py line:1
    foo	[return]	None	in /tmp/ipykernel_28345/2052304305.py line:2
  bar	[return]	0	in /tmp/ipykernel_28345/2052304305.py line:6
  recur	[call]	in /tmp/ipykernel_28345/2052304305.py line:11
    recur	[call]	in /tmp/ipykernel_28345/2052304305.py line:11
      recur	[call]	in /tmp/ipykernel_28345/2052304305.py line:11
      recur	[return]	None	in /tmp/ipykernel_28345/2052304305.py line:14
    recur	[return]	None	in /tmp/ipykernel_28345/2052304305.py line:14
  recur	[return]	None	in /tmp/ipykernel_28345/2052304305.py line:14
  error	[call]	in /tmp/ipykernel_28345/2052304305.py line:8
  error	[exception]	<class 'ZeroDivisionError'>	in /tmp/ipykernel_28345/2052304305.py line:9
  error	[return]	None	in /tmp/ipykernel_28345/2052304305.py line:9
test	[exception]	<class 'ZeroDivisionError'>	in /tmp/ipykernel_28345/29756520

ZeroDivisionError: division by zero

In [ ]:
@stack_trace_jupyter(with_return=True, with_exception=True, max_depth=3)
def test_jupyter():
    bar()
    recur(5)
    error()

test_jupyter()

test_jupyter	[call]	in /tmp/ipykernel_28345/1371165587.py line:1
  bar	[call]	in /tmp/ipykernel_28345/2052304305.py line:4
    foo	[call]	in /tmp/ipykernel_28345/2052304305.py line:1
    foo	[return]	None	in /tmp/ipykernel_28345/2052304305.py line:2
  bar	[return]	0	in /tmp/ipykernel_28345/2052304305.py line:6
  recur	[call]	in /tmp/ipykernel_28345/2052304305.py line:11
    recur	[call]	in /tmp/ipykernel_28345/2052304305.py line:11
      recur	[call]	in /tmp/ipykernel_28345/2052304305.py line:11
      recur	[return]	None	in /tmp/ipykernel_28345/2052304305.py line:14
    recur	[return]	None	in /tmp/ipykernel_28345/2052304305.py line:14
  recur	[return]	None	in /tmp/ipykernel_28345/2052304305.py line:14
  error	[call]	in /tmp/ipykernel_28345/2052304305.py line:8
  error	[exception]	<class 'ZeroDivisionError'>	in /tmp/ipykernel_28345/2052304305.py line:9
  error	[return]	None	in /tmp/ipykernel_28345/2052304305.py line:9
test_jupyter	[exception]	<class 'ZeroDivisionError'>	in /tmp/ipykerne

ZeroDivisionError: division by zero